# This Notebook Inspects the Anisotropic Classifier

The anistotropic version of the classifier uses alternating horizontal and vertical 1D convolutions instead of 2D convolutions in the decoding phase of a segnet. 

- The notebook that modifies the _facade-segnet_ net is [modify-segnet-for-anisotropy.ipynb](anisotropic-training/modify-segnet-for-anisotropy.ipynb)
- I have made a notebook to help me visualize progress as I train, if is [train_using_jupyter.ipynb](train_using_jupyter.ipynb)
- After you have funished training, this notebook will allow you to inspect the net.

---
> **NOTE** When trianing I have my data in an external, larger hard drive. This gives me enough storage to hold many snapshots of the classifier as well as preprocessed imagery for traiing. I use a soft-link so that the extranl drive appears to be part of the project. 

---
> **NOTE:** Do not try to use more than one net at a time on the same GPU, use Kernel->Restart

In [ ]:
%pylab notebook

In [ ]:
cd ..

In [ ]:
import os
sys.path.insert(0, os.path.abspath('./scripts/anisotropic-training'))

In [ ]:
#NET_WEIGHTS = 'scripts/anisotropic-training/deploy/test_weights.caffemodel'
NET_WEIGHTS = 'scripts/anisotropic-training/tn.caffemodel'
#NET_WEIGHTS='/home/shared/Projects/Facades/mybook/anisotropic/anisotropic_facades_iter_50000.caffemodel'
NET_LAYOUT = 'scripts/anisotropic-training/modified-inference-net.prototxt'

In [ ]:
import traceback

In [ ]:
import getpass
pwd = getpass.getpass('gmail:')

In [ ]:
import caffe
caffe.set_device(0)
caffe.set_mode_gpu()

In [ ]:
import warnings

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings(action='ignore')
    net = caffe.Net(NET_LAYOUT, NET_WEIGHTS, caffe.TEST)

In [ ]:
from glob import glob
files = glob('data/training/independant_12_layers/ours/npy/*.npy')

In [ ]:
index = random.choice(len(files))

In [ ]:
im = np.load(files[index])

In [ ]:
from pyfacades.util import channels_first, channels_last
figure()
imshow(channels_last(im[:3].astype(np.uint8)))

In [ ]:
result = net.forward(blobs=['prob-window'], data=array([im[:3].copy(), im[:3].copy(), im[:3].copy()]))

In [ ]:
NEGATIVE=0
POSITIVE=2
EDGE = 3

In [ ]:
from pyfacades.util import softmax

In [ ]:
softmax??

In [ ]:
blob = 'prob-window'
cw = net.blobs[blob].data[1].copy()
cw = cw[(0,2,3),...]
cw -= cw.max(axis=0)
cw = exp(cw)
cw /= np.mean(cw, axis=0)

In [ ]:
figure()
imshow(cw[1], vmin=0, vmax=1, cmap=cm.gray)
colorbar()

In [ ]:
i = 0

In [ ]:
import time
figure()
blob = 'conv-window'
net.blobs[blob].data[0][1]*=0
for i in range(net.blobs[blob].count):
    imshow(softmax(net.blobs[blob].data[0])[i], interpolation='nearest', extent=(0, 512, 0, 512), origin='upper')
    imshow(net.blobs['data'].data[0].transpose(1,2,0)/255., interpolation='nearest', extent=(0, 512, 0, 512), origin='upper', alpha=0.2)
    title(str(i))
    gcf().canvas.draw()
    time.sleep(0.5)

In [ ]:
import pyfacades.models.independant_12_layers.caffe_layers

In [ ]:
import re
solver_proto = open('scripts/anisotropic-training/solver.prototxt').read()
snapshot_prefix = re.search('^snapshot_prefix *: &*\"(.*)\"', solver_proto, re.MULTILINE).group(1).strip(' "')
print snapshot_prefix

In [ ]:
def get_iter(path):
    s = os.path.splitext(path)[0]
    s = s.rsplit('_')[-1]
    iter = int(s)
    return iter


def get_last_iter(names):
    iters = [get_iter(name) for name in names]
    max_iter = max(iters) if len(iters) > 0 else ''
    return max_iter

from glob import glob
print  get_last_iter(glob(snapshot_prefix + '*.caffemodel'))

In [ ]:
def get_weights():
    iter_ = get_last_iter(glob(snapshot_prefix + '*.caffemodel'))
    caffemodel = snapshot_prefix + '_iter_{}.caffemodel'.format(iter_)
    return caffemodel

In [ ]:
def restore_solver(solver):
    iter_ = get_last_iter(glob(snapshot_prefix + '*.caffemodel'))
    #caffemodel = snapshot_prefix + '_iter_{}.caffemodel'.format(iter_)
    solverstate = snapshot_prefix + '_iter_{}.solverstate'.format(iter_)
    print solverstate
    solver.restore(solverstate)

In [ ]:
%pushd scripts/anisotropic-training
%pwd
try:
    del net  # Delete the old net if it alrady exists
except NameError:
    pass

with warnings.catch_warnings():
    warnings.filterwarnings(action='ignore')  # Ignore warning about mpl.use
    solver = caffe.get_solver('solver.prototxt')
    #restore_solver(solver)
%popd

In [ ]:
print get_weights()
solver.net.copy_from(get_weights())

In [ ]:
#restore_solver(solver)

In [ ]:
solver.net.outputs

In [ ]:
data_layer = solver.net.layers[0]

In [ ]:
epoch_size = len(data_layer.files)
batch_size = data_layer.batch_size
iters_per_epoch = int(ceil(epoch_size/batch_size))
print "{} samples per epoch".format(epoch_size)
print "{} samples per (training) batch".format(batch_size)
print "{} iterations per epoch".format(iters_per_epoch)


In [ ]:
data_layer.verbose = False

In [ ]:
from pyfacades.util import channels_last

In [ ]:
losses = {key:[] for key in solver.net.outputs}  
losses['total'] = []

In [ ]:
from pyfacades.util import softmax

In [ ]:
import smtplib

def tell_me_about(issue):
    email_address = "femianjc@miamioh.edu"

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(email_address, pwd)

   
    msg = """\
FROM: "hal.csi.miamioh.edu" <femianjc@miamioh.edu>
SUBJECT: Deep Learning Alert

{}
.
""".format(str(issue))

    server.sendmail(email_address, email_address, msg)
    server.quit()

In [ ]:
fig = figure(figsize(10, 10))
ncols = 2
nrows = int(ceil(len(solver.net.outputs)/float(ncols)))
axes = {key:subplot(nrows, ncols, i+1) for i, key in enumerate(solver.net.outputs)}

try:
    while True:
        solver.step(1)

        assert not isnan(solver.net.params['conv1_1'][0].data.var())
        clf()

        losses['total'].append(0)
        for output in solver.net.outputs:
            losses[output].append(float(solver.net.blobs[output].data))
            losses['total'][-1] += losses[output][-1]
        
        if solver.iter % 10 != 0:
            continue

        subplot(2, 1, 1)  
        cla()
        title("Tot: {:2.4f}, Win {:2.4}, Iter {}, Epoch {}".format( 
                losses['total'][-1], losses['window-loss'][-1], solver.iter, data_layer.epochs))
        for output in ('window-loss',): #solver.net.outputs:
            xmax = len(losses['total'])
            xmin = max(0, xmax-100)
            plot(arange(xmin, xmax), losses[output][xmin:xmax])
        plot(arange(xmin, xmax), losses['total'][xmin:xmax])

        subplot(2, 2, 3)
        cla()
        imshow(channels_last(solver.net.blobs['data'].data[0])/255.)
        imshow(solver.net.blobs['facade'].data[0,0], alpha=0.3, cmap=cm.Reds)
        imshow(solver.net.blobs['window'].data[0,0], alpha=0.3, cmap=cm.Greens)
        subplot(2, 2, 4)
        cla()
        imshow(softmax(solver.net.blobs['conv-window'].data[0])[POSITIVE], cmap=cm.gray)

        fig.tight_layout()
        fig.canvas.draw()
except Exception as e:
    msg = traceback.format_exc()
    print msg
    tell_me_about(msg)

In [ ]:
solver.net.blobs['window'].data.shape

In [ ]:
def noop(*args): pass

In [ ]:
data_layer._transform = noop

In [ ]:
type(data_layer)

In [ ]:
data_layer.__class__._transform = noop